In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import random
from multiprocessing import Pool
seed_list=list(range(1,1000,100))

In [2]:
total_compounds_list=pkl.load( open("/data/dharp/compounding/datasets/total_compounds_list.pkl",'rb'))
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
modifiers, heads = zip(*novel_compounds_list)
heads_list=list(set(heads))
modifiers_list=list(set(modifiers))

## Creation of Negative Classes - CorruptHead and CorruptModifier


In [4]:
def corrupt_dataset_creator(choice):
    random.seed(choice)
    corrupt_modifier_list=[]
    corrupt_head_list=[]
    for cur_num,(modifier,head) in enumerate(novel_compounds_list):
    # CorruptHead , Modifier is same
    #print(modifier,head)

        while True:
            temp_corrupt_head=random.choice(heads_list)
            if (modifier,temp_corrupt_head) not in novel_compounds_list:
                if (modifier,temp_corrupt_head) not in total_compounds_list:
                    corrupt_head_list.append((modifier,temp_corrupt_head))
                    break
    # CorruptModifier , Head is same
        while True:
            temp_corrupt_modifier=random.choice(modifiers_list)
            if (temp_corrupt_modifier,head) not in novel_compounds_list:
                if (temp_corrupt_modifier,head) not in total_compounds_list:
                    corrupt_modifier_list.append((temp_corrupt_modifier,head))
                    break
    return corrupt_head_list,corrupt_modifier_list

In [5]:
pool = Pool(10)
results=pool.map(corrupt_dataset_creator,seed_list)

In [6]:
len(results)

10

In [9]:
for num,(corrupt_heads,corrupt_modifiers) in enumerate(results):
    print(num+1)
    pkl.dump( corrupt_heads, open( '/data/dharp/compounding/datasets/corrupt_heads_'+str(num+1)+'.pkl', "wb" ) )
    
    pkl.dump( corrupt_modifiers, open( '/data/dharp/compounding/datasets/corrupt_modifiers_'+str(num+1)+'.pkl', "wb" ) )

1
2
3
4
5
6
7
8
9
10


In [7]:
results[1][1][:20]

[('conference_n', 'peaceful_n'),
 ('constant_n', 'robinson_n'),
 ('attorney_n', 'supply_n'),
 ('extinction_n', 'peaceful_n'),
 ('reel_n', 'standard_n'),
 ('supreme_n', 'award_n'),
 ('switch_n', 'joint_n'),
 ('shoots_n', 'companion_n'),
 ('tuning_n', 'foundation_n'),
 ('familial_n', 'gentleman_n'),
 ('nothing_n', 'boat_n'),
 ('diasporic_n', 'pay_n'),
 ('scots_n', 'wife_n'),
 ('snap_n', 'othello_n'),
 ('snail_n', 'sage_n'),
 ('locality_n', 'robinson_n'),
 ('theme_n', 'evidence_n'),
 ('vermont_n', 'association_n'),
 ('advocacy_n', 'crane_n'),
 ('aol_n', 'chase_n')]

In [8]:
results[1][0][:20]

[('a_n', 'subtests_n'),
 ('a_n', 'ma_n'),
 ('a_n', 'upset_n'),
 ('india_n', 'denise_n'),
 ('india_n', 'governing_n'),
 ('india_n', 'knot_n'),
 ('india_n', 'managerial_n'),
 ('india_n', 'faith_n'),
 ('india_n', 'clause_n'),
 ('india_n', 'afield_n'),
 ('india_n', 'giles_n'),
 ('india_n', 'astrocyte_n'),
 ('india_n', 'eric_n'),
 ('india_n', 'reel_n'),
 ('india_n', 'bone_n'),
 ('the_n', 'will_n'),
 ('the_n', 'kiln_n'),
 ('the_n', 'palestine_n'),
 ('the_n', 'winner_n'),
 ('the_n', 'america_n')]